In [1]:
import json
import os 
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split



In [2]:
carpeta = '../../dataRAW/'

lista_combinada = []

for archivo in os.listdir(carpeta):
    if archivo.endswith('.json'):
        ruta_archivo = os.path.join(carpeta, archivo)  
        with open(ruta_archivo, 'r') as f:
            datos = json.load(f)
            if isinstance(datos, list): 
                lista_combinada.extend(datos) 

print(len(lista_combinada))

196917


In [3]:
ids = []
ids_vistos = set()

for x in lista_combinada:
    _id = x["id"]
    if _id not in ids_vistos:
        ids.append(_id)  # Añadir a la lista manteniendo el orden
        ids_vistos.add(_id)  # Añadir al conjunto para evitar duplicados

In [4]:
def remove_duplicates(ids, list):
    ret = []
    i = 0
    for id in ids:
        while list[i]["id"] != id:
            i = i+1
        ret.append(list[i])
    return ret

data = remove_duplicates(ids, lista_combinada)
data = [x for x in data if isinstance(x, dict)]


In [5]:
print(len(data))

172111


In [6]:
'''
bodyType None = Undetermined (can be anyone)
bodyType 1 = Berlina
bodyType 2 = Coupe
bodyType 3 = Cabrio
bodyType 4 = Familiar
bodyType 5 = Monovolumen
bodyType 6 = SUV
bodyType 7 = Pick Up
bodyType 8 = Furgoneta
'''
def assign_body_type(car):
    try:

        if not isinstance(car, dict): return None

        if isinstance(car["bodyTypeId"], str): return car["bodyTypeId"]

        typeID = car["bodyTypeId"]
        match typeID:
            case None: return None
            case 1: return "Berlina"
            case 2: return "Coupe"
            case 3: return "Cabrio"
            case 4: return "Familiar" 
            case 5: return "Monovolumen"
            case 6: return "SUV"
            case 7: return "Pick Up"
            case 8: return "Furgoneta"
        return None
    
    except KeyError:
        return None

def extraer_marca(texto):
    palabras = texto.split()  # Dividir el string por espacios
    return palabras[0]

def get_images(car):
    if not isinstance(car, dict): return None
    try:
        ret = []
        for resource in car["resources"]:
            if resource['type'] == 'IMAGE' : ret.append(resource['url'])
        return ret
    except KeyError as e:
        print("uwu la clave no existe")
        return None

In [7]:
ids = [x["id"] for x in data]
models = [assign_body_type(x) for x in data]
brand = [extraer_marca(x["title"]) for x in data]
images = [get_images(x) for x in data]

In [8]:
def extract_word(pattern, text, default_value = None):
    data = re.search(pattern, text)
    if data != None: return data.group(1)
    else: return default_value

def getJato(car):
    try: 
        text = car["detail"]["vehicleSpecs"]["vehicleStandardEquipmentGroup"][0]["equipment"][0]
        return extract_word(r'clasificación JATO: .*?(\w+\d+)', text)
    except Exception as e:
        #print(e)
        return None

In [9]:
jato = [getJato(x) for x in data]
jato.count(None)

49367

In [10]:
dataframe = pd.DataFrame({"id": ids, "model": models, "brand": brand, "images": images, "jato": jato})

print(len(dataframe))
dataframe.dropna(inplace=True, subset=['model', "brand", "images"])
print(len(dataframe))

dataframe["jato"] = jato

172111
152602


ValueError: Length of values (172111) does not match length of index (152602)

In [13]:
df_clean = dataframe.dropna(subset=['jato'])

In [14]:
encoder = OneHotEncoder()

# Transformar y convertir en matriz densa
encoded_features = encoder.fit_transform(df_clean[['model', 'brand']]).toarray()

# Crear un DataFrame con las características codificadas
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['model', 'brand']))

# Añadir la columna 'jato' al DataFrame codificado
encoded_df['jato'] = df_clean['jato'].values

# Mostrar el DataFrame codificado
#print(encoded_df)


In [15]:
# Dividir los datos en entrenamiento y prueba
train_data = encoded_df.dropna(subset=['jato'])
predict_data = encoded_df[encoded_df['jato'].isnull()]

# Separar las características y las etiquetas
X_train = train_data.drop(columns='jato')
y_train = train_data['jato']

# Modelo de clasificación
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
rf_classifier.fit(X_train, y_train)

# Preparar el conjunto de datos de predicción
X_predict = predict_data.drop(columns='jato')

# Predicción de valores faltantes
predicted_jato = rf_classifier.predict(X_predict)
predicted_jato

ValueError: Found array with 0 sample(s) (shape=(0, 67)) while a minimum of 1 is required by RandomForestClassifier.

In [34]:
dataframe["jato"].value_counts()

0

In [32]:
print(len(dataframe))
dataframe.dropna(inplace=True)
print(len(dataframe))

172111
122744


In [33]:
max_file_size = 100 * 1024 * 1024

def split_dataframe_to_csv(df, max_file_size, output_folder ,base_filename):
    file_index = 1
    chunk_size = len(df) // 7 
    while chunk_size > 0:
        start_row = 0
        while start_row < len(df):
            end_row = start_row + chunk_size
            chunk = df.iloc[start_row:end_row]
            filename = f"{output_folder}/{base_filename}_part{file_index}.csv"
            chunk.to_csv(filename, index=False)
            if os.path.getsize(filename) > max_file_size:
                chunk_size //= 2
                os.remove(filename)
            else:
                start_row = end_row
                file_index += 1

        if start_row >= len(df):
            break

split_dataframe_to_csv(dataframe, max_file_size, "photos_urls/", "photos")